In [195]:
example_rules = """
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"
"""

example_input = """
ababbb
bababa
abbbab
aaabbb
aaaabbb
"""

In [196]:
a = example_rules.strip().split('\n')
a

['0: 4 1 5',
 '1: 2 3 | 3 2',
 '2: 4 4 | 5 5',
 '3: 4 5 | 5 4',
 '4: "a"',
 '5: "b"']

In [197]:
def make_initial_rule_dict(rule_input):
    rule_dict = {}
    for rule in rule_input:
        num, rules = rule.split(': ')
        if '"' in rules:
            rule_dict[num] = [rules[1]]
    return rule_dict
rule_dict = make_initial_rule_dict(a)
rule_dict

{'4': ['a'], '5': ['b']}

In [198]:
import itertools

def resolve_lists(chars, rule_dict):
    # if any of the chars not in the dict, return
    char_list = chars.split(' ')
    for char in char_list:
        if char not in rule_dict:
            return None
        
    out = []
    strings_to_combine = [rule_dict[char] for char in char_list]
    # if any are empty, return None
    if any(len(x) == 0 for x in strings_to_combine):
        return None
    
    for element in itertools.product(*strings_to_combine):
        out.append(''.join(element))
    return out

def resolve_with_or(chars, rule_dict):
    out = []
    for expr in chars.split(' | '):
        # resolve each
        ans = resolve_lists(expr, rule_dict)
        if not ans:
            return None
        out.extend(ans)
    return out

rule_dict = make_initial_rule_dict(a)
num_rules = len(a)

while True:
    for line in a:
        num, rule = line.split(': ')
        if num in rule_dict:  # already evaluated
            continue
        # attempt to resolve rule
        resolved = resolve_with_or(rule, rule_dict)
        if resolved:
            rule_dict[num] = resolved
    if len(rule_dict) == num_rules:
        break


In [199]:
rule_dict

{'4': ['a'],
 '5': ['b'],
 '2': ['aa', 'bb'],
 '3': ['ab', 'ba'],
 '1': ['aaab', 'aaba', 'bbab', 'bbba', 'abaa', 'abbb', 'baaa', 'babb'],
 '0': ['aaaabb',
  'aaabab',
  'abbabb',
  'abbbab',
  'aabaab',
  'aabbbb',
  'abaaab',
  'ababbb']}

In [200]:
# check the received messages
messages = example_input.strip().split('\n')
count = 0
for message in messages:
    if message in rule_dict['0']:
        count += 1
count

2

In [201]:
def get_rules_messages():
    with open('inputs/input19.txt') as f:
        real_rules, real_messages = f.read().split('\n\n')
    real_rules = real_rules.split('\n')
    real_messages = real_messages.split('\n')
    return real_rules, real_messages

In [202]:
real_rules, real_messages = get_rules_messages()
    
rule_dict = make_initial_rule_dict(real_rules)
num_rules = len(real_rules)

while True:
    for line in real_rules:
        num, rule = line.split(': ')
        if num in rule_dict:  # already evaluated
            continue
        # attempt to resolve rule
        resolved = resolve_with_or(rule, rule_dict)
        if resolved:
            rule_dict[num] = resolved
    if len(rule_dict) == num_rules:
        break

In [203]:
rule_dict['42'][:5]

['baaaabba', 'babaabba', 'bbaaabba', 'bbbaabba', 'bbbbabba']

In [204]:
count = 0
allowed_set = set(rule_dict['0'])
for message in real_messages:
    if message in allowed_set:
        count += 1
count

139

In [205]:
new_rules = """
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1
"""

new_messages = """
abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
"""

new_rules = new_rules.strip().split('\n')
new_messages = new_messages.strip().split('\n')
    
rule_dict = make_initial_rule_dict(new_rules)
num_rules = len(new_rules)

# reconstruct set of substrings, but skip 8 and 11
while True:
    for line in new_rules:
        num, rule = line.split(': ')
        if num in ['8', '11', '0']:
            continue
        if num in rule_dict:  # already evaluated
            continue
        
        # attempt to resolve rule
        resolved = resolve_with_or(rule, rule_dict)
        if resolved:
            rule_dict[num] = resolved
    if len(rule_dict) == num_rules - 3:  # resolved all rules except 8, 11 and 0
        break
        
# rule 0 is now: 42k31n, for k > n and n >= 1
def process_message(message, rule_dict):
    substr_len = len(rule_dict['31'][0])
    set_42 = set(rule_dict['42'])
    set_31 = set(rule_dict['31'])
    
    chunks = [message[i:i+substr_len] for i in range(0, len(message), substr_len)]
    count = 0
    hit31 = False
    for chunk in chunks:
        if chunk in set_42:  # if yes, keep counting how many of those we've seen
            if hit31:
                return False
            count += 1
        elif chunk in set_31:  # we've hit 31, start decrementing count of 42 and don't exceed
            count -= 1
            if count <= 0:
                return False
            hit31 = True
        else:
            return False  # some other rule, no match
    return hit31  # it's a hit only if we've seen at least one 31 match

hits = set()
for message in new_messages:
    if process_message(message, rule_dict):
        hits.add(message)

print(len(hits))

12


In [206]:
hits

{'aaaaabbaabaaaaababaa',
 'aaaabbaabbaaaaaaabbbabbbaaabbaabaaa',
 'aaabbbbbbaaaabaababaabababbabaaabbababababaaa',
 'aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba',
 'ababaaaaaabaaab',
 'ababaaaaabbbaba',
 'abbbbabbbbaaaababbbbbbaaaababb',
 'baabbaaaabbaaaababbaababb',
 'babbbbaabbbbbabbbbbbaabaaabaaa',
 'bbabbbbaabaabba',
 'bbbababbbbaaaaaaaabbababaaababaabab',
 'bbbbbbbaaaabbbbaaabbabaaa'}

In [207]:
expected_hits = """
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
""".strip().split('\n')
set(expected_hits) == hits

True

In [208]:
real_rules, real_messages = get_rules_messages()

rule_dict = make_initial_rule_dict(real_rules)
num_rules = len(real_rules)

# reconstruct set of substrings, but skip 8 and 11
while True:
    for line in real_rules:
        num, rule = line.split(': ')
        if num in ['8', '11', '0']:
            continue
        if num in rule_dict:  # already evaluated
            continue
        
        # attempt to resolve rule
        resolved = resolve_with_or(rule, rule_dict)
        if resolved:
            rule_dict[num] = resolved
    if len(rule_dict) == num_rules - 3:  # resolved all rules except 8, 11 and 0
        break

hits = set()
for message in real_messages:
    if process_message(message, rule_dict):
        hits.add(message)

print(len(hits))

289
